## Code Binding summaries using Jinja2 and Python Modele

- Fetch SD file from IG
- Transform to Python model
- use Jinja2 template to create a summary markdown file
- save markdown file



### import python modules including R4 fhirclient models

In [ ]:
from fhirclient.r4models import structuredefinition as SD
from fhirclient.r4models import narrative as N
from fhirclient.r4models import valueset as VS
import fhirclient.r4models.identifier as I
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.extension as X
import fhirclient.r4models.contactdetail as CD
import fhirclient.r4models.fhirreference as FR
from fhirclient.r4models.fhirabstractbase import FHIRValidationError
from json import dumps, loads, load
from requests import get, post, put
import os
from pathlib import Path
from csv import reader as csvreader
from pprint import pprint
from stringcase import snakecase, titlecase
from collections import namedtuple
from pandas import *
from datetime import date
from jinja2 import Environment, FileSystemLoader, select_autoescape
from commonmark import commonmark
from IPython.display import display, HTML, Markdown
from lxml import etree

### Get US Core SD file names

In [37]:
canon = 'http://hl7.org/fhir/us/core/'  # ig canonical base

#in_path = '/Users/ehaas/Documents/FHIR/US-Core-R4/output/StructureDefinition-'
#f_name = 'us-core-inspired-oxygen'
ig_path = '/Users/ehaas/.fhir/packages/hl7.fhir.us.core.r4#dev/package'

ig_files = [x for x in os.listdir(ig_path) if x.startswith("StructureDefinition")]
ig_files

['StructureDefinition-us-core-careteam.json',
 'StructureDefinition-us-core-medicationstatement.json',
 'StructureDefinition-us-core-direct.json',
 'StructureDefinition-us-core-profile-link.json',
 'StructureDefinition-us-core-device.json',
 'StructureDefinition-us-core-practitioner.json',
 'StructureDefinition-us-core-patient.json',
 'StructureDefinition-us-core-immunization.json',
 'StructureDefinition-us-core-smokingstatus.json',
 'StructureDefinition-us-core-documentreference.json',
 'StructureDefinition-new-us-core-diagnosticreport.json',
 'StructureDefinition-us-core-race.json',
 'StructureDefinition-us-core-medicationrequest.json',
 'StructureDefinition-us-core-diagnosticreport.json',
 'StructureDefinition-us-core-condition.json',
 'StructureDefinition-us-core-birthsex.json',
 'StructureDefinition-us-core-encounter.json',
 'StructureDefinition-us-core-organization.json',
 'StructureDefinition-us-core-pediatric-weight.json',
 'StructureDefinition-us-core-procedure.json',
 'Struct

### Get matchin FHIR core SD file names

In [ ]:


core_path = '/Users/ehaas/.fhir/packages/hl7.fhir.core#current/package'
core_files = [x for x in os.listdir(core_path) if x.startswith("StructureDefinition")]

core_files

In [ ]:
def open_file(in_path, f_name): # get files
    with open(f'{in_path}/{f_name}') as f:
        r = f.read()
        return(loads(r))
   

In [ ]:
 
def expand(vs):
    print(vs)
    #fhir_test_server = 'http://fhirtest.uhn.ca/baseDstu3'
    #fhir_test_server = 'http://test.fhir.org/r4'
    fhir_test_server = 'https://r4.ontoserver.csiro.au/fhir'

    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }

    # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
 
    params = dict(
          url = vs.split('|')[0],
          count = 20,
            )
    
    r = get(f'{fhir_test_server}/ValueSet/$expand', params = params, headers = headers )
    # return r.status_code
    # view  output

    
    if r.status_code == 200:
        return VS.ValueSet(r.json(),strict= False)

In [53]:
j_list = []    
for i in ig_files:
    sd_dict = open_file(ig_path,i)
    try:
        core_sd_dict = open_file(core_path,i.replace("-us-core",""))
    except FileNotFoundError:
        pass
    try:
        sd = SD.StructureDefinition(sd_dict)
        diff_e = [i.path for i in sd.differential.element]
        for j in sd.snapshot.element:
            if j.min == 1 and j.binding and j.path in diff_e:
                j_url = f'{canon}{i.replace(".json","-definitions.html#")}{j.path}'
                if j.binding.strength == 'required' and j.min ==1:
                    vs = expand(j.binding.valueSet)
                    try:
                        core_sd = SD.StructureDefinition(core_sd_dict)
                        j_core = next(k for k in core_sd.snapshot.element if k.path == j.path)
                        j_core_url = f'https://build.fhir.org/{j.path.split(".")[0].lower()}-definitions.html#{j.path}'
                    except StopIteration:
                        j_core = None
                    j_list.append((j,vs,j_url,j_core, j_core_url))
    except FHIRValidationError as e:
        print(e)
    except AttributeError:
        pass 
j_list

http://hl7.org/fhir/ValueSet/care-team-status


http://hl7.org/fhir/ValueSet/medication-statement-status


http://hl7.org/fhir/ValueSet/contact-point-system


http://hl7.org/fhir/ValueSet/administrative-gender


http://hl7.org/fhir/ValueSet/immunization-status
http://hl7.org/fhir/ValueSet/observation-status


...

http://hl7.org/fhir/ValueSet/contact-point-system|4.0.0


http://hl7.org/fhir/us/core/ValueSet/us-core-narrative-status
http://hl7.org/fhir/ValueSet/request-status


http://hl7.org/fhir/ValueSet/care-plan-intent
http://hl7.org/fhir/ValueSet/goal-status


http://hl7.org/fhir/us/core/ValueSet/omb-ethnicity-category
http://hl7.org/fhir/us/core/ValueSet/detailed-ethnicity
http://hl7.org/fhir/ValueSet/allergyintolerance-verification


[(<fhirclient.r4models.elementdefinition.ElementDefinition at 0x11d726dd8>,
  'http://hl7.org/fhir/us/core/StructureDefinition-us-core-careteam-definitions.html#CareTeam.status',
  'https://build.fhir.org/careteam-definitions.html#CareTeam.status'),
 (<fhirclient.r4models.elementdefinition.ElementDefinition at 0x11d7b7710>,
  'http://hl7.org/fhir/us/core/StructureDefinition-us-core-medicationstatement-definitions.html#MedicationStatement.status',
  'https://build.fhir.org/medicationstatement-definitions.html#MedicationStatement.status'),
 (<fhirclient.r4models.elementdefinition.ElementDefinition at 0x11d7acd68>,
  'http://hl7.org/fhir/us/core/StructureDefinition-us-core-patient-definitions.html#Patient.telecom.system',
  None,
  'https://build.fhir.org/medicationstatement-definitions.html#MedicationStatement.status'),
 (<fhirclient.r4models.elementdefinition.ElementDefinition at 0x11f16e390>,
  'http://hl7.org/fhir/us/core/StructureDefinition-us-core-patient-definitions.html#Patient.ge

### Using Jinja2 Template create md file for summary view

In [54]:
k_list = []
for j in j_list:
    try:
     k_list.append(tuple(list(j) + [c.code for c in j[1].expansion.contains if c.code in ['unknown','other']]))
    except AttributeError:
        k_list.append(j)
k_list

[(<fhirclient.r4models.elementdefinition.ElementDefinition at 0x11d726dd8>,
  'http://hl7.org/fhir/us/core/StructureDefinition-us-core-careteam-definitions.html#CareTeam.status',
  'https://build.fhir.org/careteam-definitions.html#CareTeam.status'),
 (<fhirclient.r4models.elementdefinition.ElementDefinition at 0x11d7b7710>,
  'http://hl7.org/fhir/us/core/StructureDefinition-us-core-medicationstatement-definitions.html#MedicationStatement.status',
  'https://build.fhir.org/medicationstatement-definitions.html#MedicationStatement.status',
  'unknown'),
 (<fhirclient.r4models.elementdefinition.ElementDefinition at 0x11d7acd68>,
  'http://hl7.org/fhir/us/core/StructureDefinition-us-core-patient-definitions.html#Patient.telecom.system',
  None,
  'https://build.fhir.org/medicationstatement-definitions.html#MedicationStatement.status',
  'other'),
 (<fhirclient.r4models.elementdefinition.ElementDefinition at 0x11f16e390>,
  'http://hl7.org/fhir/us/core/StructureDefinition-us-core-patient-def

In [55]:
in_path = ''
template_path = 'binding_summary_template4.j2'

def markdown(text, *args, **kwargs):
    return commonmark(text, *args, **kwargs)

env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )

env.filters['markdown'] = markdown

template = env.get_template(template_path)



d = template.render(elements = k_list)
display(Markdown(d))

## All Required Bindings for Required Coded Elements:


1. **[CareTeam.status](http://hl7.org/fhir/us/core/StructureDefinition-us-core-careteam-definitions.html#CareTeam.status)**
    - [base](https://build.fhir.org/careteam-definitions.html#CareTeam.status) cardinality= 0..1
    - base valueSet = [http://hl7.org/fhir/ValueSet/care-team-status|4.0.0](http://hl7.org/fhir/ValueSet/care-team-status|4.0.0)
    - base binding strength = required
    - http://hl7.org/fhir/ValueSet/care-team-status
    (
     entered-in-error, 
     inactive, 
     active, 
     suspended, 
     proposed)
1. **[MedicationStatement.status](http://hl7.org/fhir/us/core/StructureDefinition-us-core-medicationstatement-definitions.html#MedicationStatement.status)**
    - [base](https://build.fhir.org/medicationstatement-definitions.html#MedicationStatement.status) cardinality= 1..1
    - base valueSet = [http://hl7.org/fhir/ValueSet/medication-statement-status|4.0.0](http://hl7.org/fhir/ValueSet/medication-statement-status|4.0.0)
    - base binding strength = required
    - http://hl7.org/fhir/ValueSet/medication-statement-status
    (
     completed, 
     active, 
     entered-in-error, 
     on-hold, 
     **unknown**, 
     stopped, 
     intended, 
     not-taken)
1. **[Patient.telecom.system](http://hl7.org/fhir/us/core/StructureDefinition-us-core-patient-definitions.html#Patient.telecom.system)**
    - [base](https://build.fhir.org/medicationstatement-definitions.html#MedicationStatement.status) cardinality= ..
    - http://hl7.org/fhir/ValueSet/contact-point-system
    (
     phone, 
     pager, 
     url, 
     fax, 
     other, 
     sms, 
     email)
1. **[Patient.gender](http://hl7.org/fhir/us/core/StructureDefinition-us-core-patient-definitions.html#Patient.gender)**
    - [base](https://build.fhir.org/patient-definitions.html#Patient.gender) cardinality= 0..1
    - base valueSet = [http://hl7.org/fhir/ValueSet/administrative-gender|4.0.0](http://hl7.org/fhir/ValueSet/administrative-gender|4.0.0)
    - base binding strength = required
    - http://hl7.org/fhir/ValueSet/administrative-gender
    (
     male, 
     female, 
     other, 
     **unknown**)
1. **[Immunization.status](http://hl7.org/fhir/us/core/StructureDefinition-us-core-immunization-definitions.html#Immunization.status)**
    - [base](https://build.fhir.org/immunization-definitions.html#Immunization.status) cardinality= 1..1
    - base valueSet = [http://hl7.org/fhir/ValueSet/immunization-status|4.0.0](http://hl7.org/fhir/ValueSet/immunization-status|4.0.0)
    - base binding strength = required
    - http://hl7.org/fhir/ValueSet/immunization-status
    (
     completed, 
     not-done, 
     entered-in-error)
1. **[Observation.status](http://hl7.org/fhir/us/core/StructureDefinition-us-core-smokingstatus-definitions.html#Observation.status)**
    - [base](https://build.fhir.org/immunization-definitions.html#Immunization.status) cardinality= ..
    - http://hl7.org/fhir/ValueSet/observation-status
    (
     corrected, 
     amended, 
     final, 
     registered, 
     entered-in-error, 
     preliminary, 
     cancelled, 
     **unknown**)
1. **[DocumentReference.status](http://hl7.org/fhir/us/core/StructureDefinition-us-core-documentreference-definitions.html#DocumentReference.status)**
    - [base](https://build.fhir.org/documentreference-definitions.html#DocumentReference.status) cardinality= 1..1
    - base valueSet = [http://hl7.org/fhir/ValueSet/document-reference-status|4.0.0](http://hl7.org/fhir/ValueSet/document-reference-status|4.0.0)
    - base binding strength = required
    - http://hl7.org/fhir/ValueSet/document-reference-status
    (
     entered-in-error, 
     superseded, 
     current)
1. **[DocumentReference.type](http://hl7.org/fhir/us/core/StructureDefinition-us-core-documentreference-definitions.html#DocumentReference.type)**
    - [base](https://build.fhir.org/documentreference-definitions.html#DocumentReference.type) cardinality= 0..1
    - base valueSet = [http://hl7.org/fhir/ValueSet/c80-doc-typecodes](http://hl7.org/fhir/ValueSet/c80-doc-typecodes)
    - base binding strength = preferred
    - http://hl7.org/fhir/ValueSet/c80-doc-typecodes
    (
     36325-9, 
     39337-1, 
     42473-9, 
     47522-8, 
     43458-9, 
     39920-4, 
     34751-8, 
     37393-6, 
     38019-6, 
     46349-7, 
     57177-8, 
     39670-5, 
     30688-6, 
     24659-5, 
     37764-8, 
     58744-4, 
     24909-4, 
     39336-3, 
     43791-3, 
     51392-9)
1. **[DocumentReference.content.attachment.contentType](http://hl7.org/fhir/us/core/StructureDefinition-us-core-documentreference-definitions.html#DocumentReference.content.attachment.contentType)**
    - [base](https://build.fhir.org/documentreference-definitions.html#DocumentReference.type) cardinality= ..
    - http://hl7.org/fhir/ValueSet/mimetypes|4.0.0
1. **[DiagnosticReport.status](http://hl7.org/fhir/us/core/StructureDefinition-new-us-core-diagnosticreport-definitions.html#DiagnosticReport.status)**
    - [base](https://build.fhir.org/documentreference-definitions.html#DocumentReference.type) cardinality= ..
    - http://hl7.org/fhir/ValueSet/diagnostic-report-status
    (
     registered, 
     cancelled, 
     partial, 
     entered-in-error, 
     **unknown**, 
     appended, 
     final, 
     preliminary, 
     corrected, 
     amended)
1. **[Extension.extension.valueCoding](http://hl7.org/fhir/us/core/StructureDefinition-us-core-race-definitions.html#Extension.extension.valueCoding)**
    - [base](https://build.fhir.org/documentreference-definitions.html#DocumentReference.type) cardinality= ..
    - http://hl7.org/fhir/us/core/ValueSet/omb-race-category
1. **[Extension.extension.valueCoding](http://hl7.org/fhir/us/core/StructureDefinition-us-core-race-definitions.html#Extension.extension.valueCoding)**
    - [base](https://build.fhir.org/documentreference-definitions.html#DocumentReference.type) cardinality= ..
    - http://hl7.org/fhir/us/core/ValueSet/detailed-race
1. **[MedicationRequest.status](http://hl7.org/fhir/us/core/StructureDefinition-us-core-medicationrequest-definitions.html#MedicationRequest.status)**
    - [base](https://build.fhir.org/medicationrequest-definitions.html#MedicationRequest.status) cardinality= 1..1
    - base valueSet = [http://hl7.org/fhir/ValueSet/medicationrequest-status|4.0.0](http://hl7.org/fhir/ValueSet/medicationrequest-status|4.0.0)
    - base binding strength = required
    - http://hl7.org/fhir/ValueSet/medicationrequest-status
    (
     stopped, 
     active, 
     entered-in-error, 
     cancelled, 
     completed, 
     draft, 
     **unknown**, 
     on-hold)
1. **[DiagnosticReport.status](http://hl7.org/fhir/us/core/StructureDefinition-us-core-diagnosticreport-definitions.html#DiagnosticReport.status)**
    - [base](https://build.fhir.org/diagnosticreport-definitions.html#DiagnosticReport.status) cardinality= 1..1
    - base valueSet = [http://hl7.org/fhir/ValueSet/diagnostic-report-status|4.0.0](http://hl7.org/fhir/ValueSet/diagnostic-report-status|4.0.0)
    - base binding strength = required
    - http://hl7.org/fhir/ValueSet/diagnostic-report-status
    (
     registered, 
     cancelled, 
     partial, 
     entered-in-error, 
     **unknown**, 
     appended, 
     final, 
     preliminary, 
     corrected, 
     amended)
1. **[Condition.verificationStatus](http://hl7.org/fhir/us/core/StructureDefinition-us-core-condition-definitions.html#Condition.verificationStatus)**
    - [base](https://build.fhir.org/condition-definitions.html#Condition.verificationStatus) cardinality= 0..1
    - base valueSet = [http://hl7.org/fhir/ValueSet/condition-ver-status|4.0.0](http://hl7.org/fhir/ValueSet/condition-ver-status|4.0.0)
    - base binding strength = required
    - http://hl7.org/fhir/ValueSet/condition-ver-status
    (
     differential, 
     entered-in-error, 
     unconfirmed, 
     confirmed, 
     refuted, 
     provisional)
1. **[Encounter.status](http://hl7.org/fhir/us/core/StructureDefinition-us-core-encounter-definitions.html#Encounter.status)**
    - [base](https://build.fhir.org/encounter-definitions.html#Encounter.status) cardinality= 1..1
    - base valueSet = [http://hl7.org/fhir/ValueSet/encounter-status|4.0.0](http://hl7.org/fhir/ValueSet/encounter-status|4.0.0)
    - base binding strength = required
    - http://hl7.org/fhir/ValueSet/encounter-status|4.0.0
    (
     planned, 
     **unknown**, 
     in-progress, 
     finished, 
     cancelled, 
     triaged, 
     arrived, 
     onleave, 
     entered-in-error)
1. **[Procedure.status](http://hl7.org/fhir/us/core/StructureDefinition-us-core-procedure-definitions.html#Procedure.status)**
    - [base](https://build.fhir.org/procedure-definitions.html#Procedure.status) cardinality= 1..1
    - base valueSet = [http://hl7.org/fhir/ValueSet/event-status|4.0.0](http://hl7.org/fhir/ValueSet/event-status|4.0.0)
    - base binding strength = required
    - http://hl7.org/fhir/ValueSet/event-status
    (
     completed, 
     **unknown**, 
     not-done, 
     stopped, 
     entered-in-error, 
     on-hold, 
     preparation, 
     in-progress)
1. **[Observation.status](http://hl7.org/fhir/us/core/StructureDefinition-us-core-observationresults-definitions.html#Observation.status)**
    - [base](https://build.fhir.org/procedure-definitions.html#Procedure.status) cardinality= ..
    - http://hl7.org/fhir/ValueSet/observation-status
    (
     corrected, 
     amended, 
     final, 
     registered, 
     entered-in-error, 
     preliminary, 
     cancelled, 
     **unknown**)
1. **[PractitionerRole.telecom.system](http://hl7.org/fhir/us/core/StructureDefinition-us-core-practitionerrole-definitions.html#PractitionerRole.telecom.system)**
    - [base](https://build.fhir.org/procedure-definitions.html#Procedure.status) cardinality= ..
    - http://hl7.org/fhir/ValueSet/contact-point-system|4.0.0
    (
     phone, 
     pager, 
     url, 
     fax, 
     other, 
     sms, 
     email)
1. **[CarePlan.text.status](http://hl7.org/fhir/us/core/StructureDefinition-us-core-careplan-definitions.html#CarePlan.text.status)**
    - [base](https://build.fhir.org/procedure-definitions.html#Procedure.status) cardinality= ..
    - http://hl7.org/fhir/us/core/ValueSet/us-core-narrative-status
1. **[CarePlan.status](http://hl7.org/fhir/us/core/StructureDefinition-us-core-careplan-definitions.html#CarePlan.status)**
    - [base](https://build.fhir.org/careplan-definitions.html#CarePlan.status) cardinality= 1..1
    - base valueSet = [http://hl7.org/fhir/ValueSet/request-status|4.0.0](http://hl7.org/fhir/ValueSet/request-status|4.0.0)
    - base binding strength = required
    - http://hl7.org/fhir/ValueSet/request-status
    (
     active, 
     draft, 
     completed, 
     entered-in-error, 
     **unknown**, 
     on-hold, 
     revoked)
1. **[CarePlan.intent](http://hl7.org/fhir/us/core/StructureDefinition-us-core-careplan-definitions.html#CarePlan.intent)**
    - [base](https://build.fhir.org/careplan-definitions.html#CarePlan.intent) cardinality= 1..1
    - base valueSet = [http://hl7.org/fhir/ValueSet/care-plan-intent|4.0.0](http://hl7.org/fhir/ValueSet/care-plan-intent|4.0.0)
    - base binding strength = required
    - http://hl7.org/fhir/ValueSet/care-plan-intent
    (
     proposal, 
     option, 
     plan, 
     order)
1. **[Goal.lifecycleStatus](http://hl7.org/fhir/us/core/StructureDefinition-us-core-goal-definitions.html#Goal.lifecycleStatus)**
    - [base](https://build.fhir.org/goal-definitions.html#Goal.lifecycleStatus) cardinality= 1..1
    - base valueSet = [http://hl7.org/fhir/ValueSet/goal-status|4.0.0](http://hl7.org/fhir/ValueSet/goal-status|4.0.0)
    - base binding strength = required
    - http://hl7.org/fhir/ValueSet/goal-status
    (
     active, 
     completed, 
     rejected, 
     cancelled, 
     planned, 
     entered-in-error, 
     accepted, 
     on-hold, 
     proposed)
1. **[Extension.extension.valueCoding](http://hl7.org/fhir/us/core/StructureDefinition-us-core-ethnicity-definitions.html#Extension.extension.valueCoding)**
    - [base](https://build.fhir.org/goal-definitions.html#Goal.lifecycleStatus) cardinality= ..
    - http://hl7.org/fhir/us/core/ValueSet/omb-ethnicity-category
1. **[Extension.extension.valueCoding](http://hl7.org/fhir/us/core/StructureDefinition-us-core-ethnicity-definitions.html#Extension.extension.valueCoding)**
    - [base](https://build.fhir.org/goal-definitions.html#Goal.lifecycleStatus) cardinality= ..
    - http://hl7.org/fhir/us/core/ValueSet/detailed-ethnicity
1. **[AllergyIntolerance.verificationStatus](http://hl7.org/fhir/us/core/StructureDefinition-us-core-allergyintolerance-definitions.html#AllergyIntolerance.verificationStatus)**
    - [base](https://build.fhir.org/allergyintolerance-definitions.html#AllergyIntolerance.verificationStatus) cardinality= 0..1
    - base valueSet = [http://hl7.org/fhir/ValueSet/allergyintolerance-verification|4.0.0](http://hl7.org/fhir/ValueSet/allergyintolerance-verification|4.0.0)
    - base binding strength = required
    - http://hl7.org/fhir/ValueSet/allergyintolerance-verification
    (
     confirmed, 
     refuted, 
     entered-in-error, 
     unconfirmed)

### write to file

In [56]:
out_path = ''
out_file = 'binding_summary5.md'

def write_file(path, f_name, data): # write file
    with open(f'{path}{f_name}', 'w') as f:
        f.write(data)

write_file(out_path,out_file,d)

## Fin